<a href="https://colab.research.google.com/github/itskov/intergene/blob/master/intergene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from scipy.io import mmread

# Preparing the data.
!git clone https://github.com/itskov/intergene.git

# Extracting the large data file.
! tar xvf ./intergene/data/data.tar.gz -C ./intergene/data

# Saving sparse data in mat.
mat = mmread('/content/intergene/data/GSE136049_gene_by_cell_count_matrix.txt')

# Create model dir
! mkdir './savedModel/'

In [64]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

import numpy as np

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.dense1 = Dense(1000, activation='relu')
        self.dense2 = Dense(500, activation='relu')
        self.dense3= Dense(1000, activation='relu')
        self.out = Dense(21393, activation='relu') 

    def call(self, inp):
        inp = tf.reshape(inp,[-1, 21393])
        d1 = self.dense1(inp)
        d2 = self.dense2(d1)
        d3 = self.dense3(d2)
        out = self.out(d3) 

        return out
pass
print(tf.config.list_physical_devices('GPU'))
print('Done.')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Done.


In [0]:
import tensorflow as tf
import numpy as np
def gene_generator():
  for i in range(mat.shape[1]):
    current_col = mat.getcol(i).todense()
    #current_col = current_col / np.max(current_col)

    yield (current_col, current_col)

train_data = tf.data.Dataset.from_generator(gene_generator, output_types=(tf.float32, tf.float32)).batch(50)

model = MyModel()



In [0]:
# Strat training.

# Comment this if you want to start anew.
# model.load_weights('./savedModel/model/')


optimizer = tf.keras.optimizers.Adam()
#loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

alpha = 0.0001

#@tf.function
def train_step(x_cells, y_cells):
    #print('Train!')
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)

        lin_predictions = tf.reshape(predictions, [-1])
        lin_cells = tf.reshape(y_cells, [-1])
        
        loss = 5 * tf.keras.losses.MSE(lin_predictions, lin_cells)
        
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    #tf.print(loss)
    return loss
    

In [0]:
# Instantiating the model
model = MyModel()
# model.load_weights('./savedModel/model/')

model.compile(optimizer=tf.keras.optimizers.Adam(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.CosineSimilarity())

model.fit(train_data, epochs=1)
model.save_weights('./savedModel/model/')


     72/Unknown - 3850s 53s/step - loss: -0.6966